In [13]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [14]:
# school
# hostip = "10.192.40.238"
hostip = "10.192.13.255"

# home
# hostip = "192.168.0.88"
# hostip = "192.168.0.160"
# 
# hostip = "10.192.12.188"
# hostip = "10.192.40.238"

# hostip = "10.192.41.222"

# hostip = "192.168.0.21"

topic = 'ProducerA'
# file_path = 'FIT3182-A2/data/camera_event_A.csv'
file_path = 'data/camera_event_A.csv'
# file_path = './A2/data/camera_event_A.csv' 




def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [15]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [16]:
if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Failed to connect to Kafka. Exiting..')
        exit(1)

    current_batch = None
    batch_events = []

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_row = dumps(row)
            if current_batch is None:
                current_batch = row['batch_id']

            if current_batch != row['batch_id']:
                for event in batch_events:
                    message = {
                        'event_id': event['event_id'],
                        "car_plate": event['car_plate'],
                        "camera_id": int(event['camera_id']),
                        "timestamp": event['timestamp'],
                        "speed_reading": float(event['speed_reading']),
                        "producer": "A"
                    }
                    publish_message(producer, topic, event['event_id'], dumps(message))
                producer.flush()
                print(f'Batch {current_batch} published successfully.')
                sleep(5)

                current_batch = row['batch_id']
                batch_events = []

            batch_events.append(row)

        if batch_events:
            for event in batch_events:
                message = {
                    'event_id': event['event_id'],
                    "car_plate": event['car_plate'],
                    "camera_id": int(event['camera_id']),
                    "timestamp": event['timestamp'],
                    "speed_reading": float(event['speed_reading']),
                    "producer": "A"
                }
                publish_message(producer, topic, event['event_id'], dumps(message))
            producer.flush()
            print(f'Final batch {current_batch} published successfully.')

    print('All messages published successfully.')

Publishing records..
Message published successfully. {"event_id": "d40c586c-5be6-4743-a1e3-2269d9edaa72", "car_plate": "KRN 7", "camera_id": 1, "timestamp": "2024-01-01T08:00:04", "speed_reading": 77.2, "producer": "A"}
Message published successfully. {"event_id": "85c08e3c-a0b5-45d8-a70c-df8f9a6d5829", "car_plate": "ICE 8", "camera_id": 1, "timestamp": "2024-01-01T08:00:05", "speed_reading": 103.7, "producer": "A"}
Message published successfully. {"event_id": "f5834b79-771b-4931-8da2-a5ad7f4ccd02", "car_plate": "QE 1820", "camera_id": 1, "timestamp": "2024-01-01T08:00:03", "speed_reading": 67.4, "producer": "A"}
Message published successfully. {"event_id": "d0e547bb-c4a7-4750-b7b4-8076e9b47f4f", "car_plate": "CJW 924", "camera_id": 1, "timestamp": "2024-01-01T08:00:01", "speed_reading": 148.3, "producer": "A"}
Message published successfully. {"event_id": "f3162606-1b2e-407f-951d-61d14c0a7b09", "car_plate": "CJP 278", "camera_id": 1, "timestamp": "2024-01-01T08:00:02", "speed_reading":

Message published successfully. {"event_id": "1dd988c0-d000-489b-8cf0-2cd738fb0565", "car_plate": "BQN 88", "camera_id": 1, "timestamp": "2024-01-01T08:19:48", "speed_reading": 61.3, "producer": "A"}
Message published successfully. {"event_id": "9478323b-a2f6-4592-8c9f-a9de5bb984ab", "car_plate": "ZEA 3530", "camera_id": 1, "timestamp": "2024-01-01T08:19:48", "speed_reading": 153.3, "producer": "A"}
Message published successfully. {"event_id": "3e613964-b717-44a8-bf81-a1290ae07a35", "car_plate": "SJ 15", "camera_id": 1, "timestamp": "2024-01-01T08:19:48", "speed_reading": 146.2, "producer": "A"}
Message published successfully. {"event_id": "9f538681-69f0-4efd-a3e3-ef44ee2c1094", "car_plate": "PB 55", "camera_id": 1, "timestamp": "2024-01-01T08:19:49", "speed_reading": 111.2, "producer": "A"}
Message published successfully. {"event_id": "26e1dd39-29ec-4692-bca0-5705579ed316", "car_plate": "BU 9", "camera_id": 1, "timestamp": "2024-01-01T08:19:47", "speed_reading": 93.4, "producer": "A"}

Message published successfully. {"event_id": "3b552227-5141-4478-ae2f-3290e69a10c9", "car_plate": "IMC 6788", "camera_id": 1, "timestamp": "2024-01-01T08:42:38", "speed_reading": 87.6, "producer": "A"}
Message published successfully. {"event_id": "b80e1666-e54c-408d-bd13-31907f32beb8", "car_plate": "UQ 07", "camera_id": 1, "timestamp": "2024-01-01T08:42:40", "speed_reading": 143.9, "producer": "A"}
Message published successfully. {"event_id": "d4a1869f-2472-4b79-99b9-aff566122d5a", "car_plate": "CXD 617", "camera_id": 1, "timestamp": "2024-01-01T08:42:41", "speed_reading": 75.0, "producer": "A"}
Message published successfully. {"event_id": "738b20ae-27b2-421f-aac6-7b85939e1371", "car_plate": "PJ 35", "camera_id": 1, "timestamp": "2024-01-01T08:42:39", "speed_reading": 115.9, "producer": "A"}
Message published successfully. {"event_id": "4224f70a-54a9-45b8-b2b2-6f79d2444837", "car_plate": "MC 2760", "camera_id": 1, "timestamp": "2024-01-01T08:42:41", "speed_reading": 70.6, "producer": "

Message published successfully. {"event_id": "0f2932d6-ea15-4ee9-b76c-5ddd45978e07", "car_plate": "WVU 913", "camera_id": 1, "timestamp": "2024-01-01T09:07:07", "speed_reading": 85.0, "producer": "A"}
Message published successfully. {"event_id": "f40f586f-63d3-4795-b858-26dc045d9b17", "car_plate": "HGQ 06", "camera_id": 1, "timestamp": "2024-01-01T09:07:06", "speed_reading": 87.4, "producer": "A"}
Message published successfully. {"event_id": "34991344-75d0-4a8a-8cf4-37970ff7afc5", "car_plate": "NA 632", "camera_id": 1, "timestamp": "2024-01-01T09:07:06", "speed_reading": 156.3, "producer": "A"}
Message published successfully. {"event_id": "9da727c5-c7e9-444b-884f-c7a0cfd2e859", "car_plate": "JR 506", "camera_id": 1, "timestamp": "2024-01-01T09:07:04", "speed_reading": 123.3, "producer": "A"}
Message published successfully. {"event_id": "ef6fe407-1d26-48fb-95a8-1868f05df0f3", "car_plate": "DN 1", "camera_id": 1, "timestamp": "2024-01-01T09:07:06", "speed_reading": 141.8, "producer": "A"

Message published successfully. {"event_id": "2e8b5c14-1d48-484c-8b3e-9500ee86717e", "car_plate": "HM 258", "camera_id": 1, "timestamp": "2024-01-01T09:31:22", "speed_reading": 60.2, "producer": "A"}
Message published successfully. {"event_id": "2b480838-5811-4479-b618-23f9d639eb22", "car_plate": "WG 9406", "camera_id": 1, "timestamp": "2024-01-01T09:31:22", "speed_reading": 146.0, "producer": "A"}
Message published successfully. {"event_id": "9472591a-9fef-4a05-8d77-8ce4eb57410b", "car_plate": "UWZ 88", "camera_id": 1, "timestamp": "2024-01-01T09:31:20", "speed_reading": 106.4, "producer": "A"}
Message published successfully. {"event_id": "5e764904-deaa-4d9c-871a-2da42a096726", "car_plate": "HZ 826", "camera_id": 1, "timestamp": "2024-01-01T09:31:21", "speed_reading": 68.4, "producer": "A"}
Message published successfully. {"event_id": "12247c1c-d06c-4c14-85d9-5ea85e3d86af", "car_plate": "GS 6268", "camera_id": 1, "timestamp": "2024-01-01T09:31:18", "speed_reading": 116.4, "producer": 

...

Message published successfully. {"event_id": "75f8001a-fcb6-4a8b-addf-65e70b049ed1", "car_plate": "JQ 35", "camera_id": 1, "timestamp": "2024-01-01T14:29:16", "speed_reading": 145.6, "producer": "A"}
Message published successfully. {"event_id": "43adfdc6-e678-490b-995f-0dc885d00cdd", "car_plate": "VA 7", "camera_id": 1, "timestamp": "2024-01-01T14:29:12", "speed_reading": 118.6, "producer": "A"}
Message published successfully. {"event_id": "55df66b5-0a14-4397-a499-97190e4e085c", "car_plate": "DM 245", "camera_id": 1, "timestamp": "2024-01-01T14:29:11", "speed_reading": 148.6, "producer": "A"}
Message published successfully. {"event_id": "69d3fcfe-90e3-4388-97ef-c3a73bd9f08b", "car_plate": "QD 2319", "camera_id": 1, "timestamp": "2024-01-01T14:29:13", "speed_reading": 85.6, "producer": "A"}
Message published successfully. {"event_id": "0cff7832-86bf-413a-a4e7-afb664e3b644", "car_plate": "KT 967", "camera_id": 1, "timestamp": "2024-01-01T14:29:16", "speed_reading": 105.2, "producer": "A"

Message published successfully. {"event_id": "ea8c5a67-868c-487a-968a-47f9ba2f6783", "car_plate": "KC 02", "camera_id": 1, "timestamp": "2024-01-01T14:51:31", "speed_reading": 63.7, "producer": "A"}
Message published successfully. {"event_id": "36b0f603-e762-4c64-81f2-2db5bdd0cbc1", "car_plate": "DQ 0", "camera_id": 1, "timestamp": "2024-01-01T14:51:31", "speed_reading": 152.2, "producer": "A"}
Message published successfully. {"event_id": "4e2bf5e9-8163-4c88-acb5-8aa697b29d86", "car_plate": "PV 0969", "camera_id": 1, "timestamp": "2024-01-01T14:51:31", "speed_reading": 153.4, "producer": "A"}
Message published successfully. {"event_id": "d565f9ff-8723-4209-ba4b-40091b999cb7", "car_plate": "NUR 13", "camera_id": 1, "timestamp": "2024-01-01T14:51:29", "speed_reading": 143.0, "producer": "A"}
Message published successfully. {"event_id": "444a4687-922b-4517-a3f7-807a260aeb51", "car_plate": "WC 261", "camera_id": 1, "timestamp": "2024-01-01T14:51:29", "speed_reading": 69.3, "producer": "A"}

Message published successfully. {"event_id": "f38c8535-eda7-4434-9776-9de238404975", "car_plate": "BV 2", "camera_id": 1, "timestamp": "2024-01-01T15:14:39", "speed_reading": 76.0, "producer": "A"}
Message published successfully. {"event_id": "2b82763e-d0b5-4f71-997c-a0f4cfba89ce", "car_plate": "AT 866", "camera_id": 1, "timestamp": "2024-01-01T15:14:37", "speed_reading": 114.8, "producer": "A"}
Message published successfully. {"event_id": "0a0cface-d3eb-4c7b-9fad-fd69548b8c3a", "car_plate": "PH 7", "camera_id": 1, "timestamp": "2024-01-01T15:14:41", "speed_reading": 159.1, "producer": "A"}
Message published successfully. {"event_id": "0334ce38-a749-437c-b41a-db0a310ad582", "car_plate": "DQS 7420", "camera_id": 1, "timestamp": "2024-01-01T15:14:41", "speed_reading": 80.0, "producer": "A"}
Message published successfully. {"event_id": "81571a9c-1b6a-48f1-b6d7-85badcf5f1ce", "car_plate": "WMW 9635", "camera_id": 1, "timestamp": "2024-01-01T15:14:40", "speed_reading": 69.1, "producer": "A"

Message published successfully. {"event_id": "0c9820b6-2524-4c4c-a6f4-606f087c78b3", "car_plate": "CW 48", "camera_id": 1, "timestamp": "2024-01-01T15:35:39", "speed_reading": 107.9, "producer": "A"}
Message published successfully. {"event_id": "1b92f1fa-1b9b-48f3-93ae-7e670122ffb7", "car_plate": "TN 38", "camera_id": 1, "timestamp": "2024-01-01T15:35:41", "speed_reading": 103.5, "producer": "A"}
Message published successfully. {"event_id": "1ca62ca7-a37e-4eae-a292-e8fcfe2ec613", "car_plate": "CA 4", "camera_id": 1, "timestamp": "2024-01-01T15:35:41", "speed_reading": 134.6, "producer": "A"}
Message published successfully. {"event_id": "c3ef9821-916d-40cf-9bcc-eb95a8d5d0f8", "car_plate": "YL 7", "camera_id": 1, "timestamp": "2024-01-01T15:35:39", "speed_reading": 68.4, "producer": "A"}
Message published successfully. {"event_id": "46149624-0a54-41a2-9b1b-d9baab0c89d6", "car_plate": "YNS 868", "camera_id": 1, "timestamp": "2024-01-01T15:35:41", "speed_reading": 68.1, "producer": "A"}
Me

KeyboardInterrupt: 